<a href="https://www.bigdatauniversity.com"><img src="https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width="400" align="center"></a>

<h1 align="center"><font size="5">Classification with Python</font></h1>

In this notebook we try to practice all the classification algorithms that we learned in this course.

We load a dataset using Pandas library, and apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

Lets first load required libraries:

In [ ]:
import pprint
import itertools
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import matplotlib.ticker as ticker
%matplotlib inline

### About dataset

This dataset is about past loans. The __Loan_train.csv__ data set includes details of 346 customers whose loan are already paid off or defaulted. It includes following fields:

| Field          | Description                                                                           |
|----------------|---------------------------------------------------------------------------------------|
| Loan_status    | Whether a loan is paid off on in collection                                           |
| Principal      | Basic principal loan amount at the                                                    |
| Terms          | Origination terms which can be weekly (7 days), biweekly, and monthly payoff schedule |
| Effective_date | When the loan got originated and took effects                                         |
| Due_date       | Since it’s one-time payoff schedule, each loan has one single due date                |
| Age            | Age of applicant                                                                      |
| Education      | Education of applicant                                                                |
| Gender         | The gender of applicant                                                               |

Lets download the dataset

In [ ]:
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv


### Load Data From CSV File  

In [ ]:
df = pd.read_csv('loan_train.csv')
print(df.shape)
df.head(9)


In [ ]:
# fix spelling error
df['education'] = df['education'].str.replace('Bechalor', 'Bachelor')


### Convert to date time object 

In [ ]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

In [ ]:
# inspect values
print("Possible values for each categorical key:")
for k in ['loan_status', 'education', 'Gender', 'effective_date', 'due_date']:
    vals = set(df[k])
    if type(next(iter(vals))) == pd.Timestamp:
        vals = [v.strftime('%x') for v in vals]
    print("{:13}\t {}".format(k, vals))


In [ ]:
df.describe(percentiles=[.5], include='all')


# Data visualization and pre-processing



Let’s see how many of each class is in our data set 

In [ ]:
df['loan_status'].value_counts()


260 people have paid off the loan on time while 86 have gone into collection 


Let's plot some columns to understand data better:

In [ ]:
# notice: installing seaborn might takes a few minutes
# !conda install -c anaconda seaborn -y

In [ ]:
import seaborn as sns

bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

# Pre-processing:  Feature selection/extraction

### Lets look at the day of the week people get the loan 

In [ ]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 7)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()


We see that people who get the loan at the end of the week dont pay it off, so lets use Feature binarization to set a threshold values less then day 4 

In [ ]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()

## Convert Categorical features to numerical values

Lets look at gender:

In [ ]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

86 % of female pay there loans while only 73 % of males pay there loan


Lets convert male to 0 and female to 1:


In [ ]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

## One Hot Encoding  
#### How about education?

In [ ]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)

#### Feature befor One Hot Encoding

In [ ]:
df[['Principal','terms','age','Gender','education']].head()

#### Use one hot encoding technique to conver categorical varables to binary variables and append them to the feature Data Frame 

In [ ]:
features = df[['Principal','terms','age','Gender','weekend']]
features = pd.concat([features, pd.get_dummies(df['education'])], axis=1)
features.drop(['Master or Above'], axis=1, inplace=True)
features.head()


### Feature selection

Lets defind feature sets, X:

In [ ]:
X = features
X[:5]

What are our lables?

In [ ]:
y = df['loan_status'].values
y[:5]

## Normalize Data 

Data Standardization give data zero mean and unit variance (technically should be done after train test split )

In [ ]:
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

# Classification 

Now, it is your turn, use the training set to build an accurate model. Then use the test set to report the accuracy of the model
You should use the following algorithm:
- K Nearest Neighbor(KNN)
- Decision Tree
- Support Vector Machine
- Logistic Regression



__ Notice:__ 
- You can go above and change the pre-processing, feature selection, feature-extraction, and so on, to make a better model.
- You should use either scikit-learn, Scipy or Numpy libraries for developing the classification algorithms.
- You should include the code of the algorithm in the following cells.

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics

# KNN works best with low number of dimensions
X_4 = X[:, :5]  # select columns: Principal | terms | age | Gender
X4_train, X4_test, y4_train, y4_test = train_test_split(X_4, y, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Train set:', X_train.shape, y_train.shape)
print('Test set:', X_test.shape, y_test.shape)

# K Nearest Neighbor(KNN)
Notice: You should find the best k to build the model with the best accuracy.  
**warning:** You should not use the __loan_test.csv__ for finding the best k, however, you can split your train_loan.csv into train and test to find the best __k__.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# First test
k = 4
knn = KNeighborsClassifier(n_neighbors=k, algorithm='auto').fit(X4_train, y4_train)
y4_hat = knn.predict(X4_test)
print(knn)
print("Train set Accuracy (Jaccard): {:6.4f}".format(
    metrics.accuracy_score(y4_train, knn.predict(X4_train))))
print("Test set Accuracy  (Jaccard): {:6.4f}".format(
    metrics.accuracy_score(y4_test, y4_hat)))

In [ ]:
klim = 51  # int(np.sqrt(X_train.shape[0]))
kvals = range(1, klim, 2)
acc_k = [0 for k in range(klim)]

for k in kvals:
    print("K={:02d}".format(k), end=", ")
    # Train Model and Predict  
    knn = KNeighborsClassifier(n_neighbors=k)
    fold_scores = cross_val_score(knn, X4_train, y4_train, cv=5)
    acc_k[k] = np.mean(fold_scores)

plt.plot(kvals, acc_k[1::2], color='tab:blue', label='test acc (Jaccard)')
plt.legend()
plt.show()

optimal_k = np.argmax(acc_k)
print("Optimal K={} w/ acc {:6.4f}".format(optimal_k, np.max(acc_k)))

In [ ]:
# With optimal k
k = optimal_k
knn = KNeighborsClassifier(n_neighbors=k, algorithm='auto').fit(X4_train, y4_train)
y4_hat = knn.predict(X4_test)
print(knn)
print("Train set accuracy (Jaccard): {:6.4f}".format(
    metrics.accuracy_score(y4_train, knn.predict(X4_train))))
print("Test set accuracy  (Jaccard): {:6.4f}".format(
    metrics.accuracy_score(y4_test, y4_hat)))

For the nearest neighbor analysis, only a subset of all attributes (Principal, terms, age, Gender) was used to build the model, due to the fact that KNN achieves best results when the number of dimensions is kept low.
A 5-fold cross correlation determines the optimal number of neighbors to be 27.
KNN with `k=27` achieves a Jaccard index of 0.7429, meaning roughly 3 out of 4 cases can be accurately predicted.

# Decision Tree

In [ ]:
# Notice: You might need to uncomment and install the pydotplus and graphviz libraries if you have not installed these before
# !conda install -c conda-forge pydotplus -y
# !conda install -c conda-forge python-graphviz -y

In [ ]:
from sklearn import tree

dtree = tree.DecisionTreeClassifier(criterion="entropy").fit(X_train, y_train)
y_hat = dtree.predict(X_test)

print("Train set accuracy (Jaccard): {:6.4f}".format(
    metrics.accuracy_score(y_train, dtree.predict(X_train))))
print("Test set accuracy  (Jaccard): {:6.4f}".format(
    metrics.accuracy_score(y_test, y_hat)))

In [ ]:
from six import StringIO
import pydotplus
import matplotlib.image as mpimg

dot_data = StringIO()
filename = "loan_dtree.png"
featureNames = features.columns
targetNames = df['loan_status'].unique().tolist()
out = tree.export_graphviz(dtree, feature_names=featureNames, 
                           out_file=dot_dta, class_names=np.unique(y_train), 
                           filled=True, special_characters=True, rotate=False)  

graph = pydotplus.graph_from_dot_data(dot_dta.getvalue())  
graph.write_png(filename)
img = mpimg.imread(filename)

plt.figure(figsize=(100, 200))
plt.imshow(img, interpolation='nearest')
plt.show()

In [ ]:
featureNames = features.columns
print(featureNames)
targetNames = df['loan_status'].unique().tolist()
print(targetNames)

# Support Vector Machine

# Logistic Regression

# Model Evaluation using Test set

In [ ]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

First, download and load the test set:

In [ ]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

### Load Test set for evaluation 

In [ ]:
test_df = pd.read_csv('loan_test.csv')
test_df.head()

# Report
You should be able to report the accuracy of the built model using different evaluation metrics:

| Algorithm          | Jaccard | F1-score | LogLoss |
|--------------------|---------|----------|---------|
| KNN                | ?       | ?        | NA      |
| Decision Tree      | ?       | ?        | NA      |
| SVM                | ?       | ?        | NA      |
| LogisticRegression | ?       | ?        | ?       |

<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="http://cocl.us/ML0101EN-SPSSModeler">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://cocl.us/ML0101EN_DSX">Watson Studio</a>

<h3>Thanks for completing this lesson!</h3>

<h4>Author:  <a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a></h4>
<p><a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, PhD is a Data Scientist in IBM with a track record of developing enterprise level applications that substantially increases clients’ ability to turn data into actionable knowledge. He is a researcher in data mining field and expert in developing advanced analytic methods like machine learning and statistical modelling on large datasets.</p>

<hr>

<p>Copyright &copy; 2018 <a href="https://cocl.us/DX0108EN_CC">Cognitive Class</a>. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/">MIT License</a>.</p>